In [1]:
from utils import DataManager

In [6]:
from utils import DataManager

data_manager = DataManager(
    dataset_name="cifar100",
    shuffle=True,
    seed=1992,
    init_cls=10,
    increment=10
)

In [15]:
for task in range(1, data_manager.nb_tasks):
    print(f"Task {task}")

Task 1
Task 2
Task 3
Task 4
Task 5
Task 6
Task 7
Task 8
Task 9


In [7]:
import numpy as np

train_dataset = data_manager.get_dataset(
            np.arange(0, 5),
            source="train",
            mode="train",
        )

In [2]:
import torch
from avalanche.benchmarks.utils import as_classification_dataset
from avalanche.benchmarks.scenarios.supervised import class_incremental_benchmark
from torchvision.datasets import ImageFolder
from utils import get_transform

TRAIN_TRANSFORM, TEST_TRANSFORM = get_transform(image_dim=32)

train_dataset = ImageFolder(f"data/CIFAR100/train", transform=TRAIN_TRANSFORM)
test_dataset = ImageFolder(f"data/CIFAR100/test", transform=TRAIN_TRANSFORM)

train_dataset = as_classification_dataset(train_dataset)
test_dataset = as_classification_dataset(test_dataset)

benchmark = class_incremental_benchmark({'train': train_dataset, 
                                         'test': train_dataset},
                                         num_experiences=10,
                                         seed=42)

In [37]:
exp = benchmark.train_stream[0]
print(f"Experience {exp.current_experience}")
print(f"Classes in this experience: {exp.classes_in_this_experience}")
print(f"Previous classes: {exp.classes_seen_so_far}")
print(f"Future classes: {exp.future_classes}")

Experience 0
Classes in this experience: {96, 98, 42, 78, 46, 16, 24, 60, 62, 95}
Previous classes: {96, 98, 42, 78, 46, 16, 24, 60, 62, 95}
Future classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 97, 99}


In [3]:
from avalanche.training import Naive
from models import SimCLR
from lightly.utils.benchmarking import LinearClassifier
from torchvision.models import resnet18
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import loss_metrics, accuracy_metrics, bwt_metrics, forward_transfer_metrics, topk_acc_metrics, timing_metrics, forgetting_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.logging import InteractiveLogger
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

backbone = resnet18()
backbone.fc = torch.nn.Identity()
model = SimCLR(backbone=backbone)

eval_plugin = EvaluationPlugin(topk_acc_metrics(top_k=1, epoch=True, experience=True),
                               topk_acc_metrics(top_k=5, epoch=True, experience=True),
                               loss_metrics(epoch=True, experience=True), 
                               bwt_metrics(experience=True, stream=True),
                               forward_transfer_metrics(experience=True, stream=True),
                               timing_metrics(epoch=True),
                               forgetting_metrics(experience=True, stream=True),
                               cpu_usage_metrics(experience=True),
                               confusion_matrix_metrics(num_classes=100, save_image=False, stream=True),
                               disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
                               loggers=[InteractiveLogger()],
                               strict_checks=False)

cl_strategy = Naive(model=None, optimizer=None, criterion=CrossEntropyLoss(), 
                    train_mb_size=1, train_epochs=1, eval_mb_size=32, 
                    evaluator=eval_plugin)

num_classes = 0
for train_experience, test_experience in zip(benchmark.train_stream, benchmark.test_stream):
    train_dataloader = torch.utils.data.DataLoader(train_experience.dataset, batch_size=32, shuffle=False)

    num_classes += len(train_experience.classes_in_this_experience)

    model = LinearClassifier(model=model.backbone, 
                             batch_size_per_device=32,
                             feature_dim=512,
                             num_classes=num_classes,
                             freeze_model=True)
    optimizer = AdamW(model.classification_head.parameters())

    cl_strategy.model = model
    cl_strategy.optimizer = optimizer
    cl_strategy.eval(benchmark.test_stream)
    break

-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
0it [00:00, ?it/s]

IndexError: Target 16 is out of bounds.

In [ ]:
# Step 1: Get original class names and indices (alphabetical order)
original_classes = sorted(train_experience.dataset.class_to_idx.keys()) 

original_indices = [dataset.class_to_idx[cls] for cls in original_classes]  # e.g., [0, 1, 2]

# Step 2: Create a new mapping {old_index: new_index}
old_to_new = {old_idx: new_idx for new_idx, old_idx in enumerate(original_indices)}
# Example: {0: 0, 1: 1, 2: 2} (if already consecutive)
# If original_indices were [10, 2, 5], this would map them to 0, 1, 2.

# Step 3: Update targets
new_targets = [old_to_new[old_idx] for old_idx in dataset.targets]
dataset.targets = new_targets  # Update targets

# Step 4: Update class_to_idx to reflect new indices
dataset.class_to_idx = {cls: new_idx for new_idx, cls in enumerate(original_classes)}

In [4]:
import numpy as np 

classes = np.unique(train_experience.dataset._data_attributes["targets"])
for i in range(len(train_experience.dataset._data_attributes["targets"])):
    new_class = np.where(classes==16)[0][0]  
    train_experience.dataset._data_attributes["targets"][i] = new_class

TypeError: 'DataAttribute' object does not support item assignment

In [5]:
train_experience.dataset._data_attributes

OrderedDict([('targets',
              FlatData (len=5000,subset=True,cat=False,cf=True)
              	list (len=50000))])

In [13]:
train_experience.dataset.imgs

AttributeError: 'ClassificationDataset' object has no attribute 'imgs'

In [43]:
train_experience.dataset._data_attributes.keys()

odict_keys(['targets'])